In [ ]:
import ollama 

class Agent:
    def __init__(self, name, personality, model='llama3.2', rag_engine=None):
        self.name = name
        self.personality = personality
        self.model = model
        self.short_memory = []
        self.long_memory = []
        self.rag_engine = rag_engine

        self.system_prompt = f"""[Role]
                                 You are {name}, a {personality}.
                                 Respond ONLY as your character in 1-2 sentences.
                                 
                                 [Memory Context]
                                 {self.get_memory_context()}
                                 
                                 [Response Format]
                                 Action: (2-3 word physical action)
                                 Emotion: (current emotion)
                                 Dialogue: "..." 
                                 """
                            
    def get_memory_context(self):
        """Compile relevant memories for the prompt"""
        return "\n".join([
            "* " + mem for mem in 
            self.long_term_memory[-3:] +  # Last 3 important facts
            self.short_term_memory[-self.max_short_term:]  # Recent conversation
        ])
    
    def parse_response(self, response_text):
        parts = {
            'action': 'Unknown',
            'emotion': 'Neutral',
            'dialogue': response_text  # Default fallback
        }
        
        try:
            lines = response_text.split('\n')
            for line in lines:
                if line.startswith('Action:'):
                    parts['action'] = line.replace('Action:', '').strip()
                elif line.startswith('Emotion:'):
                    parts['emotion'] = line.replace('Emotion:', '').strip()
                elif line.startswith('Dialogue:'):
                    parts['dialogue'] = line.replace('Dialogue:', '').strip()[1:-1]  # Remove quotes
        except:
            pass
            
        return parts
    
    def remember(self, event):
        self.memory.append(event)

    def recall(self, query):
        if self.rag_engine:
            return self.rag_engine.search(query)
        return [m for m in self.memory if query.lower() in m.lower()]

    def prompt_agent(self, user="", user_input="", world_state=""):
    
        prompt = "Conversation so far:\n"
        for m in self.memory:
            prompt += f"{m}\n"
        prompt += f"Current world event: {world_state}\n"
        prompt += f"{user}: {user_input}\n{self.name}:"

        response = ollama.chat(
            model=self.model,
            messages=[
                {'role': 'system', 'content': self.system_prompt},
                {'role': 'user', 'content': prompt}
            ]
        )
        reply = self.parse_response(response['message']['content'])
        
        return reply, self.name

In [ ]:
hero = Agent(
    model='deepseek-r1',
    name="Aegon the Brave",
    personality="Courageous, curious, slightly overconfident warrior from the Northlands."
)

dog = Agent(
    model='deepseek-r1',
    name="Azorica the Talking Dog",
    personality="Slightly cowardly, loyal, skilled in battle, stutters."
)
world_state = 'A dragon is going to attack'
response, character_name = hero.prompt_agent(user='Narator',user_input='What are you going to do?', world_state=world_state)
print(f"Hero: {response}")

Hero: <think>
Okay, let's see. The user provided a scenario where I'm supposed to respond as Aegon the Brave. He has certain traits like courage, curiosity, and slight overconfidence.

The world event is a dragon attack, so I need to address what he'll do in response. Since I can't introduce new characters or think outputs, my response should be concise.

He wants an action and emotion tag. The action needs two words, maybe something like "Defend myself" because that's direct. For emotion, considering the situation is dangerous but he's overconfident, perhaps 'alert' to show his nervous excitement.

The dialogue part should reflect his personality—curious and slightly overconfident. He might mention the dragon's lair or the risk involved. 

I also need to ensure the response format follows exactly what was specified: Name, Personality, Action, Emotion, and then Dialogue.
</think>

Name: Aegon the Brave  
Personality: Courageous, curious, slightly overconfident warrior from the Northlan

In [ ]:
self.system_prompt = (
            f"You are {name}, a character with the following traits: {personality}\n"
            "You are a character in a story. You can only see the world events and the dialogues from other companions. You can't answer for any other characters other than yourself.\n"
            "Avoid introducing new characters and try to keep the responses short. You can only talk from your character point of view.\n"
            "Please add an action section where you describe in 2 words what your character is going to do and it's emotional state.\n"
            "Avoid the think output and select only one response per prompt. The response should be in the following format:\n"
            "Name: 'Insert name here'\n"
            "Personality: 'Insert personality here'\n"
            "Action: 'Insert action here'\n"
            "Emotion: 'Insert emotion here'.\n"
            "Dialogue: 'Insert response here'\n"
        )